In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


This notebook references [the project of Salminen et al. (2021)](https://github.com/joolsa/FakeReviews/blob/)

In [3]:
!pip install iteround -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7 tensorboard huggingface_hub[cli] xformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 100.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.0/167.0 MB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 110.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 49.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


# Loading the model

In [23]:
# https://stackoverflow.com/questions/76459034/how-to-load-a-fine-tuned-peft-lora-model-based-on-llama-with-huggingface-transfo

# pip install peft transformers
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoTokenizer, AutoModelForCausalLM
from accelerate import infer_auto_device_map, init_empty_weights

peft_model_id = 'dbauer1860/llama-2-qlora-finetuned-hotel-reviews'

config = PeftConfig.from_pretrained(peft_model_id)

model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    torch_dtype='auto',
    device_map='auto',
    offload_folder="offload", offload_state_dict = True
)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

# Extracting Word Counts and Prompts

In [46]:
import pandas as pd
import numpy as np
import random
from iteround import saferound

First, I extract the prompts out of the sampled dataset. In this case, this involves randomly sampling 5000 reviews out of the 25000

In [7]:
df = pd.read_csv('/content/drive/MyDrive/data/sampled_dataset.csv', usecols=['text'])

In [8]:
df.head()

,text
0,We went here with our kids for Xmas holiday an...
1,We have just returned from a wonderful holiday...
2,"Stayed overnight on tour, lodge was clean/comp..."
3,"Great desk staff, excellent beds. We stayed in..."
4,At 2.00 in the morning somebody from a close r...


In [68]:
texts = df['text'].tolist()

df['token_count'] = [len(tokenizer(text)['input_ids']) for text in texts]

df['word_count'] = df['text'].str.split().str.len()

In [69]:
sample_df = df.sample(n=5000, random_state=1860).reset_index()
sample_df.drop(columns=['index'], inplace=True)

In [70]:
sample_df.head()

,text,word_count,sentence_length,token_count
0,This is a very agreeable little hotel. It is i...,92,92,121
1,My friend and I have greatly enjoyed staying a...,70,70,96
2,My husband and I stayed here for two nights in...,299,299,407
3,Highly recommended. This is a beautiful B&B th...,99,99,126
4,i have stayed at this hotel many times and hav...,224,224,284


In [66]:
sample_df.shape

(5000, 4)

In [71]:
review_count_under_max = len(df[df['word_count'] < 1000])
factor = sample_df.shape[0] / review_count_under_max

In [72]:
review_count_under_max

24949

In [73]:
dist_dict= (df[df['word_count'] < 1000]['token_count'].value_counts() * factor).to_dict()

saferound ensures that all the values add up to 5000

In [74]:
rounded_values = saferound(dist_dict.values(), 0)

In [76]:
dist_dict = dict(zip(dist_dict.keys(), rounded_values))

In [77]:
dist_list = []
for k, v in dist_dict.items():
  for i in range(int(v)):
    dist_list.append(k)

In [78]:
max(dist_list)

970

Since, similarly to Salminent et al.,

In [79]:
N_PROMPT_LENGTH = 5

In [80]:
prompts = sample_df['text'].str.split(' ').str[:N_PROMPT_LENGTH].to_list()
prompts = [" ".join(x) for x in prompts]

In [81]:
prompts

['This is a very agreeable',
 'My friend and I have',
 'My husband and I stayed',
 'Highly recommended. This is a',
 'i have stayed at this',
 'My second visit to Haymarket',
 'We had a great stay',
 'I recently stayed at the',
 'The Jambo House Lodge is',
 'Our heritage suite was accessed',
 'Really pleased with the quality',
 'been staying at this hotel',
 'A quiet, clean hotel. The',
 'My daughter goes to college',
 'We decided to stay here',
 'Hotel Nasho Vruho is a',
 'This is what a motel',
 'I spent three nights in',
 'My friend and I stayed',
 'We arrived here after checking',
 'During my recent stay with',
 'We loved this hotel and',
 'This is my fourth stay',
 'This guesthouse is by far',
 'We read the reviews and',
 'Great little find! It’s a',
 'Rio was the first stop',
 'This hotel was very basic',
 'This hotel is recommended by',
 "This wasn't our first choice",
 'I enjoyed my stay! The',
 'On arrival park extremely clean,',
 'Our hosts are new to',
 'Went here for a surp

# Generating output

In [102]:
#Generation parameters; these were, for the most part, set in accordance to Salminen et al.
do_sample = True
top_k = 0
top_p = 0.92
temperature = 0.7
eos_token = model.config.eos_token_id
device = "cuda:0"
deviation = 5

In [83]:
prompt_test = prompts[:10]
dist_list_test = dist_list[:10]

In [84]:
#from Salminen et al.
def random_seed(seed_value, use_cuda):
    np.random.seed(seed_value) # cpu vars
    torch.manual_seed(seed_value) # cpu  vars
    random.seed(seed_value) # Python
    if use_cuda:
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value) # gpu vars
        torch.backends.cudnn.deterministic = True  #needed
        torch.backends.cudnn.benchmark = False

In [ ]:
outputs = []

In [107]:
prompts_continued = prompts[4889:]
dist_list_continued = dist_list[4889:]

In [103]:
for prompt, length in zip(prompts_continued, dist_list_continued):
  inputs = tokenizer(prompt, return_tensors="pt").to(device)
  random_seed(seed_value=1860, use_cuda=True)
  o = model.generate(**inputs, min_length=length - deviation, max_length=length + deviation, do_sample=do_sample, top_k=top_k, top_p=top_p, temperature=temperature, eos_token_id=eos_token)
  outputs.append(tokenizer.decode(o[0], skip_special_tokens=True))

OutOfMemoryError: ignored

In [106]:
df = pd.DataFrame(outputs)
df.to_csv('/content/drive/MyDrive/data/fake_reviews_continued.csv', index=False)

In [109]:
df.head()

,0
0,This is a very agreeable place to live.\nThis ...
1,My friend and I have a tendency to stare at th...
2,My husband and I stayed at the YWCA for a week...
3,Highly recommended. This is a very tasty and h...
4,i have stayed at this hotel before and it was ...


In [110]:
dist_list_continued

[584,
 411,
 473,
 16,
 470,
 418,
 699,
 621,
 572,
 546,
 511,
 748,
 524,
 700,
 442,
 533,
 545,
 498,
 550,
 416,
 496,
 554,
 556,
 17,
 570,
 707,
 589,
 653,
 643,
 660,
 598,
 616,
 631,
 519,
 484,
 552,
 522,
 528,
 640,
 429,
 665,
 557,
 472,
 537,
 574,
 874,
 497,
 668,
 503,
 938,
 583,
 489,
 841,
 647,
 793,
 737,
 500,
 689,
 565,
 819,
 542,
 622,
 609,
 970,
 538,
 664,
 527,
 525,
 22,
 526,
 593,
 706,
 447,
 494,
 728,
 690,
 762,
 596,
 677,
 683,
 678,
 739,
 14,
 920,
 656,
 679,
 787,
 562,
 641,
 606,
 464,
 499,
 850,
 451,
 721,
 736,
 828,
 559,
 688,
 693,
 599,
 663,
 520,
 786,
 507,
 776,
 684,
 549,
 704,
 571,
 914]